In [1]:
import time

notebook_start_time = time.time()

Offline inference pipeline: Computing item embeddings

In [1]:
import sys
from pathlib import Path   
root_dir = str(Path().absolute().parent)

if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Adding the following directory to the PYTHONPATH: d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import features, hopsworks_integration
from recsys.config import settings

In [4]:
project, fs = hopsworks_integration.get_feature_store()

mr = project.get_model_registry()

2025-03-17 00:42:37.755 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-03-17 00:42:37,758 INFO: Initializing external client
2025-03-17 00:42:37,758 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-17 00:42:41,063 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659


In [6]:
candidate_model, candidate_features = (
    hopsworks_integration.two_tower_serving.HopsworksCandidateModel.download(mr=mr)
)

2025-03-16 00:46:00.875 | INFO     | recsys.hopsworks_integration.two_tower_serving:download:185 - Downloading 'candidate_model' version 1


In [7]:
feature_view = fs.get_feature_view(
    name="retrieval",
    version=1,
)

In [8]:
train_df, val_df, test_df, _, _, _ = feature_view.train_validation_test_split(
    validation_size=settings.TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE,
    test_size=settings.TWO_TOWER_DATASET_TEST_SPLIT_SIZE,
    description="Retrieval dataset splits",
)
train_df.head(3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (129.17s) 


,customer_id,article_id,price,month_sin,month_cos,garment_group_name,index_group_name
0,325b63f3afd49ad98305e8ca136f56d156a9782d953195...,640727001,0.050831,-0.866025,5.000000e-01,Skirts,Divided
1,5d3190aa6e12099cc77ec23b6817a9a33476b56bf963dc...,675657002,0.042356,-1.000000,-1.836970e-16,Jersey Fancy,Ladieswear
2,139711369b0211ce8ee7be62fa1f242b3d7c551963e37a...,633130002,0.016932,-0.866025,5.000000e-01,Jersey Fancy,Divided


In [9]:
item_df = features.embeddings.preprocess(train_df, candidate_features)
item_df.head(3)

,garment_group_name,article_id,index_group_name
0,Skirts,640727001,Divided
1,Jersey Fancy,675657002,Ladieswear
2,Jersey Fancy,633130002,Divided


In [10]:
embeddings_df = features.embeddings.embed(df=item_df, candidate_model=candidate_model)
embeddings_df.head()

,article_id,embeddings
0,640727001,"[-0.7041218876838684, -0.09925412386655807, -0..."
1,675657002,"[-0.2918585538864136, -0.16689883172512054, -0..."
2,633130002,"[-0.4823817312717438, -0.20597295463085175, -0..."
3,688873012,"[-0.05478645861148834, -0.10328086465597153, -..."
4,637549002,"[-0.08434338122606277, -0.13418063521385193, -..."


In [11]:
candidate_embeddings_fg = (
    hopsworks_integration.feature_store.create_candidate_embeddings_feature_group(
        fs=fs, df=embeddings_df, online_enabled=True
    )
)
logger.info("✅ Uploaded 'candidate_embeddings' Feature Group to Hopsworks!!")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213659/fs/1201283/fg/1420703


Uploading Dataframe: 100.00% |██████████| Rows 32199/32199 | Elapsed Time: 02:29 | Remaining Time: 00:00


Launching job: candidate_embeddings_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1213659/jobs/named/candidate_embeddings_1_offline_fg_materialization/executions
2025-03-16 00:53:31,837 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-03-16 00:54:29,013 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-03-16 00:54:39,143 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-03-16 00:56:27,067 INFO: Waiting for log aggregation to finish.
2025-03-16 00:57:31,894 INFO: Execution finished successfully.


2025-03-16 00:57:31.896 | INFO     | __main__:<module>:6 - ✅ Uploaded 'candidate_embeddings' Feature Group to Hopsworks!!


Expose it to the online inference pipeline as a Feature View

In [12]:
feature_view = (
    hopsworks_integration.feature_store.create_candidate_embeddings_feature_view(
        fs=fs, fg=candidate_embeddings_fg
    )
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213659/fs/1201283/fv/candidate_embeddings/version/1
